# Tokenize Markdown

In [39]:
%run ../functions.ipynb
%run ./functions.ipynb

In [ ]:
# create schema
execute_query("""
--DROP TABLE IF EXISTS mkd01;
CREATE TABLE IF NOT EXISTS mkd01 (
    hash TEXT PRIMARY KEY,
    reference TEXT,
    index NUMERIC,
    content TEXT,
    metadata JSONB,
    embedding vector(1500)
);
""")

In [ ]:
print("start")
for file in find_sources('./docs-onefront'):
    print(file)
    index_markdown(file)
print("done")

In [ ]:
print("start")
for file in find_sources('./docs-vapor'):
    print(file)
    index_markdown(file)
print("done")

In [88]:
def search(query, limit = 1):
    q1 = generate_embedding(query, dimensions=1500)
    results = execute_query("""
        SELECT
            CASE
                WHEN reference LIKE './docs-vapor%%'
                THEN 'https://development.teamsystem.com/docs/default/Component/vapor-react'
                ELSE 'https://development.teamsystem.com/docs/default/System/one-front'
            END as base_url,
            regexp_replace(regexp_replace(regexp_replace(reference, E'^\\./[^/]+', ''), E'/(index|readme)\\.md$', '', 'i'), E'\\.md$', '', 'i') as path,
            regexp_replace(REPLACE(LOWER(REGEXP_REPLACE(COALESCE(metadata->>'Header 2', metadata->>'Header 1'), E'[^a-zA-Z0-9 ]', '', 'g')), ' ', '-'), E'-+', '-', 'g') as fragment
        FROM mkd01 ORDER BY embedding <-> %s::vector LIMIT %s
""", [q1, limit]);

    # Process each row to format it correctly
    formatted_urls = []
    for row in results:
        base_url, path, fragment = row
        # Ensure there is a '/' between base_url and path if not present
        if not base_url.endswith('/') and not path.startswith('/'):
            path = '/' + path
        full_url = f"{base_url}{path}#{fragment}"
        formatted_urls.append(full_url)
    
    return formatted_urls

In [93]:
print_rows(search("i want to create a new component", 1));

https://development.teamsystem.com/docs/default/Component/vapor-react/BEST_PRACTICE#there-are-some-ways-to-create-a-new-component
